|       | Description |
| ----------- | ----------- |
| Author      | Kamin Atsavasirilert (MSFE student at UIUC)|
| Date   | 01/30/2024        |
|Email|kamina2@illinois.edu|
|Idea| Use fitted SSVI parameters from SPX index for inference |

In [24]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [18]:
def compute_theta_t(params, t):
    global kappa1, kappa2
    V, V_prime, theta = params
    theta_t = theta * t + (V - theta) * (1-np.exp(-kappa1 * t)) / kappa1 + (V_prime - theta) * kappa1 * ((1-np.exp(-kappa2 * t)) / kappa2 - (1-np.exp(-kappa1 * t)) / kappa1 ) / (kappa1 - kappa2)
    return theta_t

def compute_phi(params, t):
    global params_theta_t
    rho, eta, gamma = params
    theta_t = compute_theta_t(params_theta_t, t)
    return eta / theta_t ** (gamma)

def compute_ssvi_inference(params_w_t, params_theta_t, t, k):
    rho, eta, gamma = params_w_t
    theta_t = compute_theta_t(params_theta_t, t)
    phi = compute_phi(params_w_t, t)
    return 0.5 * theta_t * ( 1 + rho * phi * k + np.sqrt((phi * k + rho)**2 + 1 - rho **2 ))

In [19]:
tmp = pd.read_csv("./data/clean_data2023.csv", parse_dates=['QUOTE_DATE', 'EXPIRE_DATE'])
tmp

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_IV,C_BID,C_ASK,C_IV,P_price,C_price,rf,Log-Strike,div
0,2023-01-04,2023-01-04,0.00,3853.39,1000.0,0.0,0.05,8.17536,2846.8,2848.3,NaN,0.025,2847.55,0.042000,-1.348953,0.0171
1,2023-01-04,2023-01-04,0.00,3853.39,1200.0,0.0,0.05,7.06724,2646.8,2648.3,NaN,0.025,2647.55,0.042000,-1.166632,0.0171
2,2023-01-04,2023-01-04,0.00,3853.39,1400.0,0.0,0.05,6.13893,2427.6,2471.0,NaN,0.025,2449.30,0.042000,-1.012481,0.0171
3,2023-01-04,2023-01-04,0.00,3853.39,1600.0,0.0,0.05,5.33125,2246.8,2248.3,NaN,0.025,2247.55,0.042000,-0.878950,0.0171
4,2023-01-04,2023-01-04,0.00,3853.39,1800.0,0.0,0.05,4.61644,2046.8,2048.3,NaN,0.025,2047.55,0.042000,-0.761167,0.0171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453005,2023-09-29,2028-12-15,1904.04,4286.60,8800.0,2880.7,3072.60,NaN,6.3,48.4,0.13139,2976.650,27.35,0.045752,0.480588,0.0156
1453006,2023-09-29,2028-12-15,1904.04,4286.60,9200.0,3186.3,3378.20,NaN,5.0,39.1,0.13457,3282.250,22.05,0.045752,0.525040,0.0156
1453007,2023-09-29,2028-12-15,1904.04,4286.60,9600.0,3494.7,3686.60,NaN,5.0,32.7,0.13909,3590.650,18.85,0.045752,0.567600,0.0156
1453008,2023-09-29,2028-12-15,1904.04,4286.60,10000.0,3805.0,3996.90,NaN,3.1,22.3,0.13708,3900.950,12.70,0.045752,0.608422,0.0156


In [20]:
tmp_iv = tmp[(tmp['QUOTE_DATE'] == "2023-01-04")&(tmp['DTE'] > 14)]

In [25]:
x, y = np.meshgrid(np.arange(tmp_iv['Log-Strike'].min(), tmp_iv['Log-Strike'].max(), 0.02), np.arange(15/365, tmp_iv['DTE'].max()/365, 0.05))

kappa1 = 5.5
kappa2 = 0.1

################################# LIST OF PARAMETERS FOR EACH DAY #############################################

# Format: V, V_prime, theta, rho, eta, gamma => Date
# 4.196e-02  4.059e-02  1.100e-01, -0.57642764,  0.55170206,  0.6895538 => 2023-01-04
# 0.03897386, 0.03987186, 0.10999981, -0.65123362,  0.41200321,  0.74083532 => 2023-03-07
# 0.02692915, 0.0373211 , 0.10999473, -0.64939349  0.60386782  0.69145943 => 2023-05-04
# 0.01355346, 0.03302254, 0.10998425, -0.66607082,  0.53899324,  0.73193646 => 2023-08-04
# rho, eta, gamma
params_w_t = np.array([-0.57642764,  0.55170206,  0.6895538])# <= you need to manually plug in the parameters
# V, V_prime, theta
params_theta_t = np.array([4.196e-02,  4.059e-02,  1.100e-01])# <= you need to manually plug in the parameters

###############################################################################################################

wt = compute_ssvi_inference(params_w_t, params_theta_t, y, x)# This function outputs the IV from SSVI
sigma_hat = (wt / y) ** 0.5# Convert to IV
sigma_hat.shape

(99, 147)

In [27]:
fig = go.Figure(data=[go.Surface(z=sigma_hat, x=x, y=y)])
fig.update_layout(title='SSVI IV', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.update_layout(scene=dict(xaxis_title='Log-Strike', yaxis_title='DTE', zaxis_title='IV'), autosize=True)
fig.show()